In [1]:
from collections import namedtuple
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\Aditya\\CNN_CLASSIFIER'

In [4]:
DataIngestionConfig = namedtuple("DataIngestionConfig", [
    "root_dir",
    "source_URL",
    "local_data_file",
    "unzip_dir"
])

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [12]:
pip install pyyaml

   ---------------------------------------- 0.0/157.3 kB ? eta -:--:--
   -- ------------------------------------- 10.2/157.3 kB ? eta -:--:--
   ------------------------------- -------- 122.9/157.3 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 157.3/157.3 kB 1.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [13]:
import sys
from pathlib import Path

# Manually set the project path (adjust the path as per your project structure)
project_root = Path('C:/Users/Aditya/CNN_CLASSIFIER')
src_path = project_root / 'src'

# Add the src directory to the Python path
sys.path.append(str(src_path))

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


ModuleNotFoundError: No module named 'joblib'

In [14]:
pip install joblib


   ---------------------------------------- 0.0/301.8 kB ? eta -:--:--
   - -------------------------------------- 10.2/301.8 kB ? eta -:--:--
   ----------------------- ---------------- 174.1/301.8 kB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 301.8/301.8 kB 3.7 MB/s eta 0:00:00


In [15]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [17]:
import os
import urllib.request as request
from zipfile import ZipFile

In [18]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

    
    def _get_updated_list_of_files(self, list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]
    
    
    def _preprocess(self, zf: ZipFile, f: str, working_dir: str):
        target_filepath = os.path.join(working_dir, f)
        if not os.path.exists(target_filepath):
            zf.extract(f, working_dir)
        
        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)

    

    
    def unzip_and_clean(self):
        with ZipFile(file=self.config.local_data_file, mode="r") as zf:
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            for f in updated_list_of_files:
                self._preprocess(zf, f, self.config.unzip_dir)

In [19]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e

[2024-07-20 12:51:20,290: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-20 12:51:20,296: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-20 12:51:20,298: INFO: common: created directory at: artifacts]
[2024-07-20 12:51:20,299: INFO: common: created directory at: artifacts/data_ingestion]


In [ ]:
%pwd